In [1]:
!pip install torch torchvision torchaudio transformers datasets




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install transformers[torch]

   ---------------------------------------- 0.0/336.6 kB ? eta -:--:--
   -------------- ------------------------- 122.9/336.6 kB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 327.7/336.6 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 336.6/336.6 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
sudo apt-get install ffmpeg


SyntaxError: invalid syntax (2755060903.py, line 1)

In [8]:
git clone https://github.com/huggingface/transformers
cd transformers
pip install -e .


SyntaxError: invalid syntax (4211242324.py, line 1)

In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset

# Specify the delimiter as tab (\t)
dataset = load_dataset("csv", data_files={
    "train": "train.csv",
    "validation": "dev.csv",
    "test": "test.csv"
}, delimiter="\t")





Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
# No need for full paths, just use file names from the dataset
def preprocess_audio(batch):
    audio_path = batch["wav"]  # File name directly
    audio, sr = torchaudio.load(audio_path)  # Load the file
    if sr != 16000:  # Resample to 16 kHz if needed
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        audio = resampler(audio)
    batch["audio"] = audio.squeeze().numpy()
    return batch




In [5]:
import os

# List the current working directory to confirm the files are accessible
print(os.listdir("."))



['00000bdb-db88-4fa0-8a72-997005cdd9ef.wav', '003dd2f5-8535-4569-882b-cabd2f689c06.wav', '006c6ebe-d25f-41ce-9fcf-5f30faca472c.wav', '00765d45-246f-4de8-930b-c11325e9a492.wav', '008e5f22-f82d-4983-8b2e-74541c78c921.wav', '00c005fc-7103-4c89-a769-dfe038abf7fd.wav', '00d20e69-62eb-4aee-9780-6bdcc2c6d0a8.wav', '00e29b28-7bf6-4b7d-b3ca-09a7ae6efd0f.wav', '00e4da5a-82cd-4c8b-8a3d-0cdf5523f388.wav', '0145bf03-e6a2-4969-8bd6-f6d40c31bfd8.wav', '01cc8510-b8a1-4dab-bcd6-e42c12dbd0f9.wav', '01eb3a60-0972-4692-9cc4-1570d04b3436.wav', '01f4b72b-d00f-4fff-987c-b7c479c0ff95.wav', '0222bc52-2d7e-431e-a7b9-8efb6bb24ccf.wav', '023967fa-9eb6-4a6e-b8d0-1d25c7ea9b7c.wav', '02441d2b-2827-4ddb-8080-0b7cf504f9cc.wav', '0253edcf-9c6b-4f15-96dc-fb8af3abcec5.wav', '027f44ec-5cae-46c4-b6e7-55608362d053.wav', '02941783-4266-42f0-a945-f157244172b7.wav', '02aad9bb-1f88-4704-bba4-7e2e8c76dbbc.wav', '02ae1ec5-2dbe-43e3-b9ef-075feb56af50.wav', '02dd41fd-c4d4-4bf7-addd-249c65b38b72.wav', '02e4bb09-221c-4b8a-9a74-938d59

In [6]:
import torchaudio

# Test loading one file
test_file = "clipping-306514b0-d498-41ff-b6ef-333cc36045d8.wav"
waveform, sample_rate = torchaudio.load(test_file)
print(f"Waveform shape: {waveform.shape}, Sample rate: {sample_rate}")



Waveform shape: torch.Size([1, 94947]), Sample rate: 16000


In [7]:
# Verify the dataset structure
print(dataset["train"].column_names)


['wav', 'words', 'duration']


In [8]:
import torchaudio

def preprocess_audio(batch):
    # Load the audio file directly from the current folder
    audio_path = batch["wav"]
    audio, sr = torchaudio.load(audio_path)
    
    # Resample to 16 kHz if necessary
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        audio = resampler(audio)
    
    batch["audio"] = audio.squeeze().numpy()
    return batch

# Apply preprocessing to the dataset
dataset = dataset.map(preprocess_audio)



Map:   0%|          | 0/6827 [00:00<?, ? examples/s]

Map:   0%|          | 0/2252 [00:00<?, ? examples/s]

Map:   0%|          | 0/2267 [00:00<?, ? examples/s]

In [9]:
from transformers import WhisperProcessor

# Load Whisper processor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

def preprocess_transcription(batch):
    # Tokenize audio features and transcription
    batch["input_features"] = processor(batch["audio"], sampling_rate=16000).input_features[0]
    batch["labels"] = processor.tokenizer(batch["words"]).input_ids
    return batch

# Apply transcription preprocessing
dataset = dataset.map(preprocess_transcription, remove_columns=["audio", "words", "duration"])


c:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

c:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--openai--whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/6827 [00:00<?, ? examples/s]

Map:   0%|          | 0/2252 [00:00<?, ? examples/s]

Map:   0%|          | 0/2267 [00:00<?, ? examples/s]

In [27]:
dataset.save_to_disk("preprocessed_dataset")


Saving the dataset (0/14 shards):   0%|          | 0/6827 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2252 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2267 [00:00<?, ? examples/s]

In [28]:
import os

# Check the 'train' directory locally
local_dataset_path = "preprocessed_dataset/train"
print(os.listdir(local_dataset_path))


['data-00000-of-00014.arrow', 'data-00001-of-00014.arrow', 'data-00002-of-00014.arrow', 'data-00003-of-00014.arrow', 'data-00004-of-00014.arrow', 'data-00005-of-00014.arrow', 'data-00006-of-00014.arrow', 'data-00007-of-00014.arrow', 'data-00008-of-00014.arrow', 'data-00009-of-00014.arrow', 'data-00010-of-00014.arrow', 'data-00011-of-00014.arrow', 'data-00012-of-00014.arrow', 'data-00013-of-00014.arrow', 'dataset_info.json', 'state.json']


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://conda.anaconda.org:443 "GET /conda-forge/notices.json HTTP/1.1" 404 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/notices.json HTTP/1.1" 404 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/notices.json HTTP/1.1" 404 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/notices.json HTTP/1.1" 404 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS conn

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\HP\anaconda3

  added / updated specs:
    - accelerate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    accelerate-0.32.1          |     pyhd8ed1ab_0         203 KB  conda-forge
    ca-certificates-2024.12.14 |       h56e8100_0         154 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    conda-23.7.4               |  py311h1ea47a8_0         1.3 MB  conda-forge
    conda-content-trust-0.2.0  |     pyhd8ed1ab_0          57 KB  conda-forge
    cssselect-1.2.0            |     pyhd8ed1ab_0          21 KB  conda-forge
    fsspec-2023.3.0            |     pyhd8e

In [25]:
!pip uninstall accelerate huggingface_hub -y
!pip install accelerate huggingface_hub


Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
Found existing installation: huggingface-hub 0.27.1
Uninstalling huggingface-hub-0.27.1:
  Successfully uninstalled huggingface-hub-0.27.1
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)
Using cached huggingface_hub-0.27.1-py3-none-any.whl (450 kB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import huggingface_hub

print(f"HuggingFace Hub version: {huggingface_hub.__version__}")


HuggingFace Hub version: 0.23.0
